# Data Science Explore Clean Analyze Workflow

Let's say I received a .csv file from a friend who's quite interested in fitness but isn't much for analyzing data (in reality said friend is a badass at analytics but that wouldn't make for as fun a story.) She asks me to have a look at the file and tell her interesting results. Rather than explain there's a lot more that needs to happen before the results point, I assure her I'll find things. 

## Explore

The first thing to do is get the file read into my environment. It's a .csv so let's see what the first few lines look like:

In [1]:
! head ./p3Roos.csv

Ok then, looks like the first line is something other than column headers or data rows. Conveniently, it starts with a #, I'll assume any line starting with # is a comment. With that in mind, let's get this file read in.

In [2]:
import pixiedust
from pyspark.sql.types import *

Pixiedust database opened successfully


In [4]:
df=spark.read.csv(path='./p3Roos.csv',comment='#',header=True, inferSchema=True,timestampFormat='HH:mm:ss:SSS')

I allowed Spark to infer the file's schema. It's a good idea to check the results of this inference. Can do so as follows:

In [5]:
df.printSchema()

root
 |-- nSmpNo: integer (nullable = true)
 |-- Day: string (nullable = true)
 |-- Mo: string (nullable = true)
 |-- Yr: double (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- RawSecs: integer (nullable = true)
 |-- RawMilli: integer (nullable = true)
 |-- Class: integer (nullable = true)
 |-- Dlat: integer (nullable = true)
 |-- DLon: integer (nullable = true)
 |-- DHeight: integer (nullable = true)
 |-- MechPwr: integer (nullable = true)
 |-- Speed: integer (nullable = true)
 |-- North: double (nullable = true)
 |-- West: double (nullable = true)
 |-- Up: double (nullable = true)
 |-- Hdng: double (nullable = true)
 |-- PodStatI: integer (nullable = true)
 |-- PodStatII: integer (nullable = true)
 |-- Hr: integer (nullable = true)
 |-- Jump Height: integer (nullable = true)
 |-- TotDist: integer (nullable = true)
 |-- MPower(w): integer (nullable = true)



I don't want the Yr to be a double, time to fix that. 

In [6]:
df = df.withColumn('Yr', df['Yr'].cast(IntegerType()))

Ok, first up, Heartrate, I can get some statistics on heart rate for the session and that should already be interesting. Here we go:

In [7]:
df.describe(['Hr']).show()

+-------+------------------+
|summary|                Hr|
+-------+------------------+
|  count|             39307|
|   mean|181.32258885185846|
| stddev| 75.87756529174032|
|    min|                60|
|    max|               255|
+-------+------------------+



My friend is *not* pleased with these results. As the max HR alone has her questioning the veracity of my interesting results. So, back to it I go, time to check out the data in a bit more detail.

For the tabular minded, Pixiedust provides another means for viewing the first few data rows, as well as viewing the DataFrame schema via the display() function. This in turn conveniently leads into a further exploration effort, plotting. 

In [8]:
display(df)

nSmpNo,Day,Mo,Yr,Time,RawSecs,RawMilli,Class,Dlat,DLon,DHeight,MechPwr,Speed,North,West,Up,Hdng,PodStatI,PodStatII,Hr,Jump Height,TotDist,MPower(w)
0,Friday,May 26,2017,1970-01-01 09:24:58.200000,1385112298,200,255,-110,-1816,4379,71,18,-4.388,7.244,3.276,91.5,0,10,255,6,0,813
1,Friday,May 26,2017,1970-01-01 09:24:58.300000,1385112298,300,255,-112,-1777,4379,79,19,1.624,7.748,3.716,91.5,0,10,255,4,0,920
2,Friday,May 26,2017,1970-01-01 09:24:58.400000,1385112298,400,255,-113,-1768,4386,80,19,2.632,7.688,3.772,90.0,0,10,255,4,0,933
3,Friday,May 26,2017,1970-01-01 09:24:58.500000,1385112298,500,255,-112,-1739,4386,94,20,7.988,8.036,4.528,90.0,0,10,255,5,1,1120
4,Friday,May 26,2017,1970-01-01 09:24:58.600000,1385112298,600,255,-114,-1730,4389,88,19,7.84,7.884,4.168,90.0,0,10,255,5,1,1040
5,Friday,May 26,2017,1970-01-01 09:24:58.700000,1385112298,700,255,-112,-1710,4389,80,18,4.22,7.512001,3.74,90.0,0,10,255,5,1,933
6,Friday,May 26,2017,1970-01-01 09:24:58.800000,1385112298,800,255,-110,-1674,4379,80,18,1.332,7.316,3.744,90.0,0,10,255,4,1,933
7,Friday,May 26,2017,1970-01-01 09:24:58.900000,1385112298,900,255,-108,-1655,4379,74,16,-5.232,6.448,3.448,90.0,0,10,255,6,1,853
8,Friday,May 26,2017,1970-01-01 09:24:59,1385112299,0,255,-109,-1606,4353,72,16,-4.836,6.492,3.324,82.5,0,10,255,6,1,826
9,Friday,May 26,2017,1970-01-01 09:24:59.100000,1385112299,100,255,-109,-1585,4353,65,16,-3.108,6.48,2.964,82.5,0,10,255,5,2,733


Check out the heart rate column (HR), that's a lot of 255 values. Red flag alert! Let's plot HR against the sample number column (nSmpNo):

In [9]:
display(df)

Oh no! Look at that! We have upper and lower rails. Clearly we'll need to be mindful of this not handling the textbook healthy HR range.

Had I done further analysis of my friend's heart rate w.r.t. other metrics, I would have been using data that indicates she had a much higher heart rate than actual. Take this several steps further and we've now entered the realm of Fitbit's class action lawsuit from 2016.

## Clean

To resolve this circumstance, we can create a new DataFrame only containing records wherein HR is > 60 and < 255, as follows:

In [10]:
cleanerDF = df.filter(df['Hr']<255)
cleanerDF = cleanerDF.filter(cleanerDF['Hr']>60)

In [11]:
cleanerDF.describe(['Hr']).show()

+-------+------------------+
|summary|                Hr|
+-------+------------------+
|  count|             17836|
|   mean|132.86286162816776|
| stddev|  43.4514294410181|
|    min|                61|
|    max|               233|
+-------+------------------+



As a result, my friend is much happier with this average heart rate, though she did bug me for more interesting results, which I'll get to in another notebook.